In [ ]:
import re
import string
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
import nltk
from nltk.corpus import stopwords
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
import pickle
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import pyLDAvis.gensim
import warnings
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import numpy as np 
import pandas as pd
import string
import re
from tqdm._tqdm_notebook import tqdm_notebook,tnrange,tqdm
from collections import Counter,OrderedDict
from gensim import models,corpora
from gensim.summarization import summarize,keywords
import warnings
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis.gensim
import gensim.models.phrases as gen
from gensim.models.coherencemodel import CoherenceModel

import preprocessor as p

In [ ]:
def preprocess(df):
    testing = df['full_text']
    cleaned = []
    p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.EMOJI, 	p.OPT.NUMBER)
    for i in testing:
        cleaned.append(p.clean(i))
    df['Clean_Tweet'] = pd.Series(cleaned)
    df['Clean_Tweet'] = df['Clean_Tweet'].astype(str)
    
    return df

In [ ]:
January = "JSONs/1.json"
February = "JSONs/2.json"
March = "JSONs/3.json"
April = "JSONs/4.json"
May = "JSONs/5.json"
June = "JSONs/6.json"
July = "JSONs/7.json"
August = "JSONs/8.json"
September = "JSONs/9.json"
October = "JSONs/10.json"
November = "JSONs/11.json"
December = "JSONs/12.json"

df1 = preprocess(pd.read_json(January))[["created_at","Clean_Tweet"]]
df2 = preprocess(pd.read_json(February))[["created_at","Clean_Tweet"]]
df3 = preprocess(pd.read_json(March))[["created_at","Clean_Tweet"]]
df4 = preprocess(pd.read_json(April))[["created_at","Clean_Tweet"]]
df5 = preprocess(pd.read_json(May))[["created_at","Clean_Tweet"]]
df6 = preprocess(pd.read_json(June))[["created_at","Clean_Tweet"]]
df7 = preprocess(pd.read_json(July))[["created_at","Clean_Tweet"]]
df8 = preprocess(pd.read_json(August))[["created_at","Clean_Tweet"]]
df9 = preprocess(pd.read_json(September))[["created_at","Clean_Tweet"]]
df10 = preprocess(pd.read_json(October))[["created_at","Clean_Tweet"]]
df11 = preprocess(pd.read_json(November))[["created_at","Clean_Tweet"]]
df12 = preprocess(pd.read_json(December))[["created_at","Clean_Tweet"]]

year_months = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12]

In [ ]:
year = pd.concat(year_months)

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
for i in stopwords.fileids():
    if i != 'english':
        stop_words.extend(stopwords.words(i))
stop_words.extend(["amp", "u"])

def words(text):
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    text = regex.sub(" ", text.lower())
    words = text.split(" ")
    words = [re.sub('\S*@\S*\s?', '', sent) for sent in words]
    words = [re.sub('\s+', ' ', sent) for sent in words]
    words = [re.sub("\'", "", sent) for sent in words]
    words = [w for w in words if not len(w) < 2]
    words = [w for w in words if w not in stop_words]
    words = [lmtzr.lemmatize(w) for w in words]
    
    return words

In [ ]:
nltk.download('wordnet')
docs = [words(x) for x in year['Clean_Tweet']]

[nltk_data] Downloading package wordnet to /home/souparno/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 103024
Number of unique words after removing rare and common words: 9768


In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 5);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=dictionary,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=500,
                                            passes=20,
                                            alpha='auto',
                                            per_word_topics=True)

In [ ]:
get_lda_topics(lda_model, 10)

Topic # 01 Topic # 02 Topic # 03 Topic # 04 Topic # 05 Topic # 06  \
0       need   pandemic    vaccine     people   lockdown     corona   
1     health    country        get       time       home      virus   
2   american      still       mask        say       stay        got   
3    million      state       know      would       work       bill   
4     relief       good       wear         go      check       died   

  Topic # 07   Topic # 08 Topic # 09 Topic # 10  
0        new  coronavirus      trump       like  
1        day     hospital       make       week  
2       case    president        let      since  
3        one       family   everyone       back  
4       year        casos       stop       last

In [ ]:
topics = lda_model.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.030*"need" + 0.026*"health" + 0.025*"american" + 0.022*"million"')
(1, '0.134*"pandemic" + 0.027*"country" + 0.027*"still" + 0.023*"state"')
(2, '0.066*"vaccine" + 0.043*"get" + 0.043*"mask" + 0.025*"know"')
(3, '0.054*"people" + 0.022*"time" + 0.020*"say" + 0.017*"would"')
(4, '0.075*"lockdown" + 0.044*"home" + 0.028*"stay" + 0.024*"work"')
(5, '0.035*"corona" + 0.023*"virus" + 0.018*"got" + 0.014*"bill"')
(6, '0.047*"new" + 0.038*"day" + 0.038*"case" + 0.038*"one"')
(7, '0.074*"coronavirus" + 0.019*"hospital" + 0.016*"president" + 0.015*"family"')
(8, '0.082*"trump" + 0.033*"make" + 0.026*"let" + 0.023*"everyone"')
(9, '0.049*"like" + 0.029*"week" + 0.023*"since" + 0.023*"back"')


In [ ]:
x = [[0.03,]
[0.026,]
[0.025,]
[0.022,]]

y =pd.toDataframe(x)
 print(y)